<a href="https://colab.research.google.com/github/ghadaalhabib/Supportive-Code/blob/main/Code_to_convert_bounding_boxes_in_Pascal_VOC_format_to_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os

In [ ]:
# Mount the google drive
from google.colab import drive 

In [ ]:
# Import necessary libraries
import pandas as pd
import os

In [ ]:
# Read csv files containing
os.chdir('/content/drive/My Drive/dataset')
classes_data = pd.read_csv('classes.csv', header=None)
classes_data.head()
print(classes_data)


     0                  1
0    0           GRAFFITI
1    1      FADED_SIGNAGE
2    2           POTHOLES
3    3            GARBAGE
4    4  CONSTRUCTION_ROAD
5    5     BROKEN_SIGNAGE
6    6    BAD_STREETLIGHT
7    7      BAD_BILLBOARD
8    8       SAND_ON_ROAD
9    9   CLUTTER_SIDEWALK
10  10      UNKEPT_FACADE


In [ ]:
# Define required classes
classes = ['GRAFFITI', 'FADED_SIGNAGE', 'POTHOLES', 'GARBAGE', 'CONSTRUCTION_ROAD' , 'BROKEN_SIGNAGE', 'BAD_STREETLIGHT', 'BAD_BILLBOARD', 'SAND_ON_ROAD', 'CLUTTER_SIDEWALK', 'UNKEPT_FACADE']
print(classes)

['GRAFFITI', 'FADED_SIGNAGE', 'POTHOLES', 'GARBAGE', 'CONSTRUCTION_ROAD', 'BROKEN_SIGNAGE', 'BAD_STREETLIGHT', 'BAD_BILLBOARD', 'SAND_ON_ROAD', 'CLUTTER_SIDEWALK', 'UNKEPT_FACADE']


In [ ]:
from numpy import require
# Get the strings corresponding to the class name and store it inside a loist called as class_strings
class_strings = []
for i in classes:
  # Extract the first column's elements for matching classes
  req_classes = classes_data.loc[classes_data[1] == i]
  string = req_classes.iloc[0][0]
  #stringg = str(string)
  #print(stringg)
  class_strings.append(string)
print(class_strings)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
 # Get only required columns from annotations csv file and get rid of not required columns

 annotation_data = pd.read_csv('train.csv', usecols=['image_path', 'class', 'xmin', 'xmax', 'ymin', 'ymax'])
 annotation_data.head()

,class,image_path,xmax,xmin,ymax,ymin
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,797.0,701.0,262.0,211.0
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,932.0,786.0,329.0,238.0
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,736.0,657.0,275.0,229.0
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,986.0,786.0,136.0,0.0
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,667.0,549.0,228.0,179.0


In [ ]:
# Add new columns that are required for YOLO format

annotation_data['classNumber'] = ''
annotation_data['center x'] = ''
annotation_data['center y'] = ''
annotation_data['width'] = ''
annotation_data['height'] = ''

annotation_data.head()


,class,image_path,xmax,xmin,ymax,ymin,classNumber,center x,center y,width,height
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,797.0,701.0,262.0,211.0,,,,,
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,932.0,786.0,329.0,238.0,,,,,
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,736.0,657.0,275.0,229.0,,,,,
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,986.0,786.0,136.0,0.0,,,,,
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,667.0,549.0,228.0,179.0,,,,,


In [ ]:
# Iterate through all the class strings and assign a class number according to the order they are appearing in the list

for i in range(len(class_strings)):
  # Store the result of this operation into classNumber
  annotation_data.loc[annotation_data['class'] == class_strings[i], 'classNumber'] = i


In [ ]:
# Calculate center x and center y values
annotation_data['center x'] = ((annotation_data['xmax'] + annotation_data['xmin']) / (2*1920))
annotation_data['center y'] = ((annotation_data['ymax'] + annotation_data['ymin']) / (2*1080))


In [ ]:
# Calculate width and height values
annotation_data['width'] = ((annotation_data['xmax'] - annotation_data['xmin'])/1920)
annotation_data['height'] = ((annotation_data['xmax'] - annotation_data['xmin'])/1080)


In [ ]:
# Generate the dataframe with YOLO required values
YOLO_values = annotation_data.loc[:,['image_path','classNumber', 'center x', 'center y','width','height']].copy()
YOLO_values.head()

In [ ]:
os.chdir('/content/drive/My Drive/dataset/images')

# Go through all the image files in the directory 
# Fullstop in os.walk('.') means the current directory
for current_dir, dirs, files in os.walk('.'):
  for f in files:
    # Check if the file extension ends with '.jpg'
    if f.endswith('.jpg'):
      # Extract only the title of the image file so that corresponding txt file can be generated with the same name as per YOLO guidelines
      #image_title = f[:-4]

      # Get matched data
      #YOLO_file = YOLO_values.loc[YOLO_values['image_path'] == image_title]
      YOLO_file = YOLO_values.loc[YOLO_values['image_path'] == f]


      # Create a copy of the above dataframe to avoid any mess
      df = YOLO_file.loc[:, ['classNumber', 'center x', 'center y', 'width', 'height']].copy()
      image_title = f[:-4]
      save_path = '/content/drive/My Drive/dataset/images' + '/' + image_title + '.txt'

      # Generate a text file containing required data in YOLO format
      df.to_csv(save_path, header=False, index=False, sep=' ')